In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import seaborn as sns
import pandas as pd
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import r2_score

sns.set()

## Import des données et cleaning

In [4]:
df = pd.read_csv("//data/03_primary/sales_transaction.csv ")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '//data/03_primary/sales_transaction.csv '

In [3]:
type(df.Date)
df[df.isna() == True].count()

TransactionNo    0
Date             0
ProductNo        0
ProductName      0
Price            0
Quantity         0
CustomerNo       0
Country          0
dtype: int64

In [4]:
type(df.Date)

pandas.core.series.Series

In [5]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Date_day'] = df.Date.dt.day
df['Date_month'] = df.Date.dt.month
df['Date_year'] = df.Date.dt.year

In [6]:
df.Date_year.value_counts()
df = df[df.Date_year == 2018]

In [7]:
df = pd.get_dummies(df, columns=['Date_year'], drop_first=True, prefix='Date_year')
df = pd.get_dummies(df, columns=['Date_month'], drop_first=True, prefix='Date_month')
df = pd.get_dummies(df, columns=['Date_day'], drop_first=True, prefix='Date_day')
df = pd.get_dummies(df, columns=['ProductName'], drop_first=True, prefix='ProductName')
df = pd.get_dummies(df, columns=['Country'], drop_first=True, prefix='Country')

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(['Date','Price','TransactionNo','ProductNo'], axis=1),
                                                    df['Price'], test_size=0.25)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

In [9]:
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42) #the random_state parameter is used for initializing the internal random number generator, which will decide the splitting of data into train and test indices in your case.
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)


[15:08:12] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [10]:
mse=mean_squared_error(y_pred, y_test)
print(np.sqrt(mse))

2.77664717975247


In [11]:
xgb_model.score(X_test, y_test) #(accuracy is #correct_preds / #all_preds

0.9433342078992742

In [12]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.9433342078992742

In [13]:
#Minimisation de la fonction de perte


def xgb_evaluate(max_depth, gamma, colsample_bytree, nrounds):
    params = {'eval_metric': 'rmse',
              'objective':'reg:squarederror',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, xgb_model, num_boost_round=int(nrounds), nfold=3, metrics=['rmse'])    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -cv_result['test-rmse-mean'].iloc[-1]

In [14]:
xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9),
                                             'nrounds': (100, 1000)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=1, n_iter=25, acq='ei')

|   iter    |  target   | colsam... |   gamma   | max_depth |  nrounds  |
-------------------------------------------------------------------------


AttributeError: 'XGBRegressor' object has no attribute 'get_uint_info'

In [ ]:
print(r2_score(y_test, xgb_model.predict(X_test)))